In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install matplotlib tensorflow==1.15.4 numpy inflect librosa scipy Unidecode pillow torch==1.9.1 tensorboardX

In [ ]:
! git clone https://github.com/funnylittleman/Cross-Lingual-Voice-Cloning
%cd Cross-Lingual-Voice-Cloning
! git submodule init; git submodule update
%cd waveglow
!  git pull origin master; git checkout 6188a1d106a1060336040db82f464d6441f39e21
! gdown --id 1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx
%cd ..

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence

from encoder import inference as encoder
from encoder.params_model import model_embedding_size as speaker_embedding_size

from model import Tacotron2

In [ ]:
# %cd ./waveglow
# waveglow_path = base_dir+'<waveglow-weights-path>'                              # To get universal vocoder waveglow weights--> https://drive.google.com/open?id=1okuUstGoBe_qZ4qUEF8CcwEugHP7GM_b
# new_model_path = base_dir+'<waveglow-new-weights-path>'
# !python convert_model.py '<waveglow-wts-old-path>'  '<waveglow-wts-new-path>'
# %cd ..

# waveglow = torch.load(new_model_path)['model']
# waveglow.cuda().eval() #.half()
# for k in waveglow.convinv:
#     k.float()
#denoiser = Denoiser(waveglow)

waveglow_path = 'waveglow/waveglow_256channels_ljs_v2.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
# denoiser = Denoiser(waveglow)

In [ ]:
hparams = create_hparams()
checkpoint_path = '/content/drive/My Drive/Cross-Lingual-Voice-Cloning/checkpoint_1000'
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval() #.half()
speaker_encoder = encoder.load_model('encoder/pretrained.pt')

In [ ]:
text = "Could there be a talking plot"
audiopath = '000005.wav'
lang = 'ru' # ['zh', 'de', 'nl', 'fr', 'ru']


lang = ['zh', 'de', 'nl', 'fr', 'ru'].index(lang)
sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()
preprocessed_wav = encoder.preprocess_wav(audiopath)
speaker_emb = encoder.embed_utterance(preprocessed_wav).cuda()
lang = torch.tensor(lang, device='cuda:0')
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence, speaker_emb, lang)
print(mel_outputs.shape)

In [ ]:
import IPython.display as ipd
with torch.no_grad():
    audio = waveglow.infer(mel_outputs, sigma=0.1)                                    #PUT MEL IN MEL_OUTPUTS_POSTNET
print(audio.shape)
ipd.Audio(audio[0].data.cpu().numpy(), rate=22050)